In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests

import numpy as np
import pandas as pd
import time
import re 

from IPython.display import Image

#Word cloud
# from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image # 만약 "No module named 'PIL'" 에러가 발생하면 [ pip install Pillow==5.4.1 ] 로 라이브러리를 설치해줍니다.
from nltk.corpus import stopwords

1-1) SongID 크롤링


In [2]:
# 노래 고유 번호(song id) 가져오기 
def get_songid(chrome_path, gnrcode):
    driver = webdriver.Chrome(executable_path=chrome_path) 
    
    # 노래 고유번호
    song_id = []
    
    # url의 startindex 변경 -> Page 변경 (10페이지, 장르당 500곡)
    for start_idx in range(1,102,50): # 1,452,50
        songlist_url = 'https://www.melon.com/genre/song_list.htm?gnrCode=' + gnrcode \
        + '&dtlGnrCode=#params%5BgnrCode%5D=' + gnrcode \
        + '&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj' \
        + '&startIndex=' + str(start_idx)

        print(songlist_url)

        driver.get(songlist_url)
        time.sleep(3)
        web = driver.page_source
        source = BeautifulSoup(web,'html.parser')

        # 노래 고유번호인 Songid를 추출
        for song_info in source.find_all('div',{'class':'ellipsis rank01'}):
            # print('크롤링 중입니다 : ', song_info.find('a')['title'][:-2])
            id = song_info.find('a')['href']
            id = re.findall("\d+", id)[-1]  
            song_id.append(id)
        print('****다음 페이지로 넘어갑니다****')

    # 하나의 장르 크롤링 완료
    print("---------------------------------------------------")
    print("장르 랩 크롤링 완료되었습니다")

    driver.close()
    driver.quit()
    
    return song_id

In [3]:
def songid_to_info(songlist):
    #변수 담을 리스트들 생성
    titles = []
    artists = []
    dates = []
    genres = []
    lyrics = []

    #곡 정보 받아오기
    idx=0
    for songid in songlist:
        url = "https://www.melon.com/song/detail.htm?songId="+ songid
        
        # print(url)

        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web = requests.get(url,headers=headers).content
        web_page = BeautifulSoup(web, 'html.parser')
        
        # print(web_page)

        #노래 제목 줄바꿈 제거
        title = web_page.find('div',{'class':'song_name'}).get_text().strip().split('\t')[-1]
        if title:
            #title = title.split('(')[0].strip() # (feat.) 등 제거
            titles.append(title)
        else:
            titles.append(' ')
        
        #가수명
        artist = web_page.find('a',{'class':'artist_name'})['title']
        if artist:
            artists.append(artist)
        else:
            artists.append(' ')
            
        #곡 발매일, 장르
        date_genre = web_page.find('dl',{'class':'list'}).get_text()
        if date_genre:
            date = date_genre.split('\n')[4]
            genre = date_genre.split('\n')[6]
            dates.append(date)
            genres.append(genre)
        else:
            dates.append(' ')
            genres.append(' ')

        #가사
        try: 
            lyric = web_page.find('div',{'class':'lyric'}).get_text().strip()
            if lyrics:
                lyrics.append(lyric)
            else:
                lyrics.append(' ')
        except: # 성인인증이 필요한 경우 스크래핑 불가 -> 가사 생략
            print('----{} : 성인인증이 필요한 가사입니다----'.format(title))
            lyrics.append(' ')
            

        idx += 1 #다음곡으로 넘어가기
        if idx%100 ==0:
            print(idx, '번째곡 크롤링 완료 :', title)

    info_dict = {'Title':titles, 
                 'Artist':artists, 
                 'Date':dates, 
                 'Genre':genres, 
                 'Lyric':lyrics}
    print(len(titles), len(artists), len(dates), len(genres), len(lyrics))
    df_song_info = pd.DataFrame(info_dict)
    return df_song_info

In [4]:
# 개인 크롬 path 삽입 
chrome_path = '/opt/homebrew/bin/chromedriver'

# 맡은 장르 적어주세요 
genre_name = '발라드'

In [5]:
# 장르와 장르 id 매칭 
gnr_info = {'발라드':'GN0100', '댄스':'GN0200', '랩/힙합':'GN0300', 'R&B/Soul':'GN0400',
           '인디음악':'GN0500', '록/메탈':'GN0600', '트로트':'GN0700','포크/블루스':'GN0800'}

# song_id 가져오기 
song_id = get_songid(chrome_path, gnr_info[genre_name])

# 노래 정보 가져오기 
df_genre_songid_to_info = songid_to_info(song_id)

# # 엑셀 반환 
# filename = "{0}_500songs.xlsx".format(genre_name) 
# df_genre_songid_to_info.to_excel(filename, index=False, encoding='utf-8')

/var/folders/rz/3h16f8bd5s36k1xv36vgkl1m0000gn/T/ipykernel_74049/2719708897.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path)


https://www.melon.com/genre/song_list.htm?gnrCode=GN0100&dtlGnrCode=#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=1
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0100&dtlGnrCode=#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=51
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0100&dtlGnrCode=#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=101
****다음 페이지로 넘어갑니다****
---------------------------------------------------
장르 랩 크롤링 완료되었습니다


AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
df_genre_songid_to_info

In [ ]:
df_genre_songid_to_info.Lyric.iloc[1]

In [ ]:
test = "      안녕하세요 \t\t      그런데요\t\t 아니요       "
test2 = "\t\t dkdldljdah \t\t"

In [ ]:
test.strip()

In [ ]:
test.strip().split('\t')

In [ ]:
test.strip().split('\t')[-1]